<a href="https://colab.research.google.com/github/gurovic/MLCourse/blob/main/135_perceptron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Перцептрон и однослойные нейронные сети

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score

## Идея перцептрона: биологическая аналогия

Перцептрон, разработанный Фрэнком Розенблаттом в 1957 году, был вдохновлен биологическими нейронами. Рассмотрим аналогию:

| **Биологический нейрон**       | **Искусственный нейрон**       |
|-------------------------------|--------------------------------|
| Дендриты (входные сигналы)    | Входные признаки (x₁, x₂, ...)|
| Сома (тело нейрона)           | Сумматор + функция активации   |
| Аксон (выходной сигнал)       | Выход (0 или 1)                |
| Синапсы (соединения)          | Веса (w₁, w₂, ...)             |

## Математическая модель перцептрона

Формально перцептрон описывается:

1. **Входной вектор**: $x = [x_1, x_2, ..., x_n]$
2. **Веса модели**: $w = [w_1, w_2, ..., w_n]$
3. **Функция активации** (ступенчатая):
   
   $$
   f(z) =
   \begin{cases}
   1 & \text{если } z \geq 0 \\
   0 & \text{иначе}
   \end{cases}
   $$

4. **Выход**: $y = f(w \cdot x + b)$, где $b$ — смещение (bias)

In [ ]:
def step_function(z):
    return 1 if z >= 0 else 0

def perceptron_predict(x, weights, bias):
    z = np.dot(weights, x) + bias
    return step_function(z)

## Алгоритм обучения перцептрона

Процедура обучения (правило Розенблатта):

1. Инициализировать веса малыми случайными значениями
2. Для каждого обучающего примера:
   - Вычислить выход $y_{pred}$
   - Обновить веса:
     $$
     w_i = w_i + \eta (y_{true} - y_{pred}) x_i
     $$
     $$
     b = b + \eta (y_{true} - y_{pred})
     $$
   где $\eta$ — скорость обучения (learning rate)

In [ ]:
class SimplePerceptron:
    def __init__(self, learning_rate=0.01, n_iters=100):
        self.lr = learning_rate
        self.n_iters = n_iters
        self.weights = None
        self.bias = None

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features)
        self.bias = 0

        for _ in range(self.n_iters):
            for idx, x_i in enumerate(X):
                linear_output = np.dot(x_i, self.weights) + self.bias
                y_pred = step_function(linear_output)

                update = self.lr * (y[idx] - y_pred)
                self.weights += update * x_i
                self.bias += update

    def predict(self, X):
        linear_output = np.dot(X, self.weights) + self.bias
        return np.array([step_function(z) for z in linear_output])

## Практическое применение

### Линейно разделимые данные

In [ ]:
# Генерация данных
X, y = make_classification(
    n_samples=100,
    n_features=2,           # Общее количество признаков
    n_informative=1,        # Информативные признаки
    n_redundant=1,          # Избыточные признаки (линейные комбинации информативных)
    n_repeated=0,           # Дублирующие признаки (точные копии других)
    n_classes=2,
    n_clusters_per_class=1,
    random_state=42)



# Обучение перцептрона
perceptron = SimplePerceptron(learning_rate=0.1, n_iters=100)
perceptron.fit(X, y)

# Визуализация
plt.figure(figsize=(10,6))
plt.scatter(X[:,0], X[:,1], c=y, cmap='cool')

# Граница решения
x_min, x_max = X[:,0].min()-1, X[:,0].max()+1
y_min, y_max = X[:,1].min()-1, X[:,1].max()+1
xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.01),
                     np.arange(y_min, y_max, 0.01))
Z = perceptron.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)
plt.contourf(xx, yy, Z, alpha=0.3)
plt.title("Граница решения перцептрона")
plt.show()

### Сравнение с sklearn Perceptron

In [ ]:
from sklearn.linear_model import Perceptron

sk_perceptron = Perceptron(max_iter=100, eta0=0.1, random_state=42)
sk_perceptron.fit(X, y)

print("Наша реализация:", accuracy_score(y, perceptron.predict(X)))
print("Sklearn реализация:", accuracy_score(y, sk_perceptron.predict(X)))

## Ограничения перцептрона

**Теорема о сходимости перцептрона** (Новиков, 1962):
Если данные линейно разделимы, перцептрон гарантированно найдет разделяющую гиперплоскость за конечное число шагов.

Однако:
1. Не работает с нелинейно разделимыми данными (например, XOR)
2. Чувствителен к масштабу данных
3. Дает только линейные границы решений

In [ ]:
# Пример с данными XOR (не решается перцептроном)
X_xor = np.array([[0,0], [0,1], [1,0], [1,1]])
y_xor = np.array([0, 1, 1, 0])

xor_perceptron = SimplePerceptron(n_iters=1000)
xor_perceptron.fit(X_xor, y_xor)
print("Точность на XOR:", accuracy_score(y_xor, xor_perceptron.predict(X_xor)))

## Вариации и модификации

1. **Адалин (ADALINE)** - использует линейную функцию активации и минимизирует MSE
2. **Многослойный перцептрон** - базовый строительный блок для нейронных сетей
3. **Перцептрон с другими функциями активации** (сигмоид, ReLU)

In [ ]:
class Adaline:
    def __init__(self, learning_rate=0.01, n_iters=100):
        self.lr = learning_rate
        self.n_iters = n_iters
        self.weights = None
        self.bias = None

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features)
        self.bias = 0

        for _ in range(self.n_iters):
            for idx, x_i in enumerate(X):
                linear_output = np.dot(x_i, self.weights) + self.bias
                error = y[idx] - linear_output
                self.weights += self.lr * error * x_i
                self.bias += self.lr * error

    def predict(self, X):
        return np.where(np.dot(X, self.weights) + self.bias >= 0, 1, 0)

## Практические рекомендации

1. **Масштабирование данных**:
```python
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
```

2. **Подбор скорости обучения**:
   - Слишком большой η → колебания весов
   - Слишком маленький η → медленная сходимость

3. **Критерии остановки**:
   - Максимальное число итераций
   - Достижение заданной точности
   - Отсутствие изменений весов

## Упражнения

1. Реализуйте перцептрон с сигмоидной функцией активации
2. Сравните скорость сходимости при разных η
3. Примените перцептрон к датасету iris (два класса)
4. Визуализируйте изменение весов в процессе обучения

In [ ]:
# Пример решения задания 3
from sklearn.datasets import load_iris
iris = load_iris()
X_iris = iris.data[:100, :2]  # Берем только setosa и versicolor
y_iris = iris.target[:100]

perceptron_iris = SimplePerceptron(n_iters=100)
perceptron_iris.fit(X_iris, y_iris)
print("Точность на Iris:", accuracy_score(y_iris, perceptron_iris.predict(X_iris)))